<a href="https://colab.research.google.com/github/Teeba73/Audio-classifier-cluster/blob/main/Audio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install webrtcvad

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp311-cp311-linux_x86_64.whl size=73498 sha256=a343b48dc6e53c366e3879d4c8a72f34b9dfd9002cdc7e33dfad8735eb8e9795
  Stored in directory: /root/.cache/pip/wheels/94/65/3f/292d0b656be33d1c801831201c74b5f68f41a2ae465ff2ee2f
Successfully built webrtcvad


In [ ]:
import os  # للتعامل مع الملفات والمجلدات
import tarfile  # لفك ضغط الملفات
import librosa  # لتحليل الصوت واستخراج الميزات
import numpy as np  # للحسابات الرياضية
import pandas as pd  # لمعالجة البيانات
import webrtcvad  # لكشف الصوت البشري (إزالة الضوضاء)
from tqdm import tqdm  # لعرض شريط التقدم
from sklearn.model_selection import train_test_split  # لتقسيم البيانات
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix  # لتقييم النموذج
import matplotlib.pyplot as plt  # لرسم الرسوم البيانية
import soundfile as sf  # لقراءة الملفات الصوتية
from xgboost import XGBClassifier  # خوارزمية التصنيف
from sklearn.preprocessing import StandardScaler  # لتوحيد البيانات
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score
)#Classification Matrics

# Configuration
TAR_PATH = "/content/cv-corpus-19.0-delta-2024-09-13-en.tar.gz"  # مسار الملف المضغوط
EXTRACT_TO = "/content/cv-corpus-19.0-delta-2024-09-13"  # مجلد فك الضغط
SNR_THRESHOLD = 15  # الحد الفاصل بين الصوت النظيف والضجيج (dB)
vad = webrtcvad.Vad(2)  # كاشف الصوت البشري (مستوى العدوانية: 0-3)

# Create output directories
#os.makedirs("train", exist_ok=True)
#os.makedirs("test", exist_ok=True)

In [ ]:
def extract_dataset():
    if not os.path.exists(TAR_PATH):
        print(f"Error: Dataset file not found at {TAR_PATH}")
        # You might want to raise an exception or exit here
        return
    # Optional: Add a check for file size if you know the expected size
    # import os
    # expected_size_bytes = ... # Get the expected size from the dataset source
    # if os.path.getsize(TAR_PATH) < expected_size_bytes:
    #     print(f"Error: Dataset file is likely incomplete. Size is {os.path.getsize(TAR_PATH)} bytes, expected at least {expected_size_bytes} bytes.")
    #     return

    if not os.path.exists(EXTRACT_TO):  # إذا لم يتم فك الضغط من قبل
        print("Extracting dataset...")
        try:
            with tarfile.open(TAR_PATH, "r:gz") as tar:
                tar.extractall(path=os.path.dirname(EXTRACT_TO))  # فك الضغط
            print("Extraction complete.")
        except tarfile.ReadError as e:
            print(f"Error reading tar file: {e}")
            print("The dataset file might be corrupted or incomplete. Please re-download it.")
        except EOFError as e:
            print(f"EOFError during extraction: {e}")
            print("The dataset file ended unexpectedly. It might be incomplete or corrupted. Please re-download it.")
        except Exception as e:
            print(f"An unexpected error occurred during extraction: {e}")
    else:
        print("Dataset already extracted")  # إذا كان موجودًا مسبقًا

In [ ]:
def extract_mfcc(file_path):
    y, sr = librosa.load(file_path, sr=None)  # تحميل الملف الصوتي
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # استخراج 13 معامل MFCC
    delta = librosa.feature.delta(mfcc)  # حساب المشتقة الأولى (لتتبع التغيرات)
    delta2 = librosa.feature.delta(mfcc, order=2)  # المشتقة الثانية (لتسارع التغيرات)
    return np.concatenate([np.mean(mfcc.T, axis=0), np.mean(delta.T, axis=0), np.mean(delta2.T, axis=0)])  # دمج الميزات

In [ ]:
def compute_snr(y):
    signal_power = np.mean(y ** 2)  # قوة الإشارة
    noise = y - librosa.effects.remix([1], intervals=librosa.effects.split(y))  # عزل الضوضاء
    noise_power = np.mean(noise ** 2) if len(noise) > 0 else 1e-9  # قوة الضوضاء
    return 10 * np.log10((signal_power + 1e-9) / (noise_power + 1e-9))  # SNR بالديسيبل

In [ ]:
def is_speech_vad(y, sr):
    if sr != 16000:
        y = librosa.resample(y, orig_sr=sr, target_sr=16000)  # تحويل معدل العينة إلى 16kHz
    y_pcm = (y * 32767).astype(np.int16).tobytes()  # تحويل الصوت إلى تنسيق PCM
    frame_size = int(16000 * 0.03) * 2  # حجم الإطار (30ms)
    num_frames = len(y_pcm) // frame_size
    speech_frames = sum(vad.is_speech(y_pcm[i*frame_size:(i+1)*frame_size], 16000) for i in range(num_frames))
    return speech_frames > 0.4 * num_frames  # إذا كان 40% من الإطارات صوت بشري

In [ ]:
def load_all_data(root_path, snr_threshold=15):
    features, labels, langs = [], [], []
    # Added a check to ensure dir_name is a directory
    for dir_name in os.listdir(root_path):
        dir_path = os.path.join(root_path, dir_name)
        if os.path.isdir(dir_path):  # Check if it's a directory
            tsv_path = os.path.join(dir_path, "validated.tsv")  # ملف البيانات
            clips_path = os.path.join(dir_path, "clips")  # ملفات الصوت
            # Also check if the required files/directories exist within the language folder
            if os.path.exists(tsv_path) and os.path.exists(clips_path):
                print(f"Processing language directory: {dir_name}")
                try:
                    df = pd.read_csv(tsv_path, sep="\t")  # قراءة البيانات
                    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {dir_name} clips"):
                        # Added a check if the 'path' column exists and if the file exists
                        if 'path' in row and isinstance(row['path'], str):
                             file_path = os.path.join(clips_path, row["path"])
                             if os.path.exists(file_path):
                                y, sr = sf.read(file_path)  # قراءة الصوت
                                if not is_speech_vad(y, sr):  # تخطي إذا لم يكن صوتًا بشريًا
                                    continue
                                try:
                                    snr = compute_snr(y)  # حساب SNR
                                    label = 0 if snr >= snr_threshold else 1  # 0=نظيف، 1=مزعج
                                    mfcc = extract_mfcc(file_path)  # استخراج الميزات
                                    features.append(mfcc)
                                    labels.append(label)
                                except Exception as e:
                                    print(f"Error processing file {file_path}: {e}")
                             else:
                                 print(f"Clip file not found: {file_path}")
                        else:
                             print(f"Invalid or missing 'path' in row for {dir_name}")
                except FileNotFoundError:
                    print(f"Validated TSV not found in {dir_path}")
                except pd.errors.EmptyDataError:
                     print(f"Validated TSV is empty in {dir_path}")
                except Exception as e:
                    print(f"Error processing directory {dir_path}: {e}")
            else:
                print(f"Skipping {dir_path}: Missing validated.tsv or clips directory.")
        else:
            # Print a message for non-directory entries found in the root path
            print(f"Skipping non-directory entry: {dir_name} found in {root_path}")

    # Add a check if any data was loaded
    if not features:
        print("No data loaded. Please check the dataset path and structure.")

    return np.array(features), np.array(labels)

In [ ]:
def train_and_evaluate(X, y):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Feature Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Train XGBoost
    clf = XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        eval_metric='logloss',
        use_label_encoder=False
    )
    clf.fit(X_train, y_train)

    # Predictions
    y_pred = clf.predict(X_test)
    y_proba = clf.predict_proba(X_test)[:, 1]  # Probabilities for ROC-AUC

    # --- Metrics ---
    print("\n Classification Metrics:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
    print(f"ROC-AUC: {roc_auc_score(y_test, y_proba):.4f}")

    # Detailed Report
    print("\n Classification Report:")
    print(classification_report(y_test, y_pred, target_names=['Clean', 'Noisy']))

    # Confusion Matrix
    print("\n Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Plot Confusion Matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(confusion_matrix(y_test, y_pred),
                annot=True, fmt='d',
                cmap='Blues',
                xticklabels=['Clean', 'Noisy'],
                yticklabels=['Clean', 'Noisy'])
    plt.title("Confusion Matrix")
    plt.show()

In [ ]:
# Replace the content of ipython-input-9-972361fa1b80 with the following:

def main():
    # Extract the dataset
    extract_dataset()

    # Define the path to the extracted dataset root
    # Assuming you want to process the English language data.
    # Verify this path matches the actual extracted directory structure.
    dataset_root = os.path.join(EXTRACT_TO, "en")
    print(f"Attempting to load data from: {dataset_root}")

    # Check if the dataset_root directory exists
    if not os.path.exists(dataset_root):
        print(f"Error: Dataset root directory not found at {dataset_root}. Please check the extraction path and dataset structure.")
        # Exit or handle the error appropriately if the directory doesn't exist
        return

    # Load the data (features and labels)
    features, labels = load_all_data(dataset_root, snr_threshold=SNR_THRESHOLD)

    print(f"Finished loading data. Number of samples loaded: {len(features)}")

    # Check if any data was loaded before training
    if len(features) == 0:
        print("No data was loaded after filtering. Cannot train the model.")
        print("Please check the dataset contents, the VAD filter, and the SNR threshold.")
    else:
        # Train and evaluate the model
        train_and_evaluate(features, labels)

if __name__ == "__main__":
    main()

Error: Dataset file not found at /content/cv-corpus-19.0-delta-2024-09-13-en.tar.gz
Attempting to load data from: /content/cv-corpus-19.0-delta-2024-09-13/en
Error: Dataset root directory not found at /content/cv-corpus-19.0-delta-2024-09-13/en. Please check the extraction path and dataset structure.
